### **Objectives**

1. 실습명 : Knowledge Distillation을 위한 LLM 기반 Pseudo labeled 데이터 생성
2. 핵심 주제:
    1. API 셋팅
    2. 데이터 불러오기
    3. 프롬프트 엔지니어링
    4. LLM을 이용한 Pseudo Labeling
3. 학습 목표 :
    1. Teacher 모델(LLM)로 사용할 Upstage Solar를 셋팅할 수 있다.
    2. 미리 저장된 데이터셋을 불러올 수 있다.
    3. LLM을 이용한 Knowledge Distillation을 위해 프롬프트 엔지니어링을 진행할 수 있다.
    4. LLM을 이용하여 Pseudo Labeling을 할 수 있다.
4. 학습 개념: 키워드명 :
    1. LLM
    2. Prompt Engineering
    3. Pseudo Labeling
5. 학습 방향 :
  - 로컬에서 돌리기 어려운 parameter size의 거대 모델을 API를 이용해서 불러와 Pseudo Labeling을 진행하여 학습 데이터를 준비합니다.
  - 실습 코드는 조교가 직접 구현한 코드를 참고하여 학습합니다.
  - 해당 실습은 데이터가 있긴 하지만 정답 데이터가 없는 상황을 어떻게 하면 해결할 수 있을지 고민합니다.


### **Prerequisites**
```
dotenv==0.9.9
openai==1.100.0
pandas==2.2.3
```


# 0. Knowledge Distillation(지식 증류, KD) 방법론

우리가 지금 실습하려는 **Pseudo Labeling**은 KD의 가장 기본적인 형태 중 하나입니다. 하지만 학계와 현업에서는 Teacher 모델의 '어떤 지식'을 '어떻게' 빼내느냐에 따라 다양한 방법론이 존재합니다. 이를 크게 전달하는 지식의 형태(Knowledge Type)에 따라 세 가지로 분류하고, 최근 **LLM 시대의 방법론**을 별도로 짚어보겠습니다.

### **1. Response-Based Knowledge (결과 중심 증류)**

가장 전통적이고(Geoffrey Hinton, 2015), 지금도 가장 많이 쓰이는 방식입니다. Teacher의 **최종 출력값(Last Layer Output)**을 Student가 모방하게 합니다.

* **Soft Targets (Logits Matching):**
  * **개념:** 단순히 정답(Hard Label, [0, 1, 0])만 배우는 게 아니라, 오답이지만 가능성 있는 확률(Soft Label, [0.05, 0.9, 0.05])까지 배웁니다.
  * **핵심:** `Temperature (T)` 파라미터를 사용하여 확률 분포를 부드럽게(Softening) 만듭니다. 이를 통해 Student는 "이 사진은 강아지지만, 고양이와도 꽤 비슷하구나"라는 **Dark Knowledge(숨겨진 지식)**를 학습합니다.


* **Hard Targets (Pseudo Labeling):**
  * **개념:** Teacher가 뱉은 가장 높은 확률의 클래스를 정답(Ground Truth)으로 간주하고 학습합니다.
  * **특징:** 우리가 진행할 **이번 실습**이 바로 여기에 해당합니다. Teacher(API)의 Logit(확률값)에 접근할 수 없는 **Black-box** 환경에서 가장 유용한 방법입니다.



### **2. Feature-Based Knowledge (중간 과정 증류)**

Teacher 모델이 깊어질수록 최종 출력값보다는 **중간 레이어(Intermediate Layers)**에 더 풍부한 특징(Feature) 정보가 담겨 있다는 점에 착안한 방법입니다.

* **Hint Learning / Feature Map Matching:**
  * **개념:** Teacher의 중간 레이어에서 나오는 Feature Map과 Student의 중간 레이어 Feature Map이 유사해지도록 손실 함수(Loss Function)를 설계합니다.
  * **엔지니어링 이슈:** Teacher와 Student의 레이어 크기(Dimension)가 다르기 때문에, 이를 맞춰주기 위한 추가적인 **Regressor(변환 레이어)**가 필요합니다.


* **Attention Transfer:**
  * **개념:** Teacher가 입력 데이터의 '어떤 부분'을 중요하게 보고 있는지(Attention Map)를 Student에게 전수합니다. "이 문제를 풀 때 여기를 봐야 해"라고 시선을 교정해 주는 것과 같습니다.



### **3. Relation-Based Knowledge (관계 중심 증류)**

데이터 하나하나의 출력값을 모방하는 것을 넘어, 데이터들 사이의 **관계(Relation)**를 학습합니다.

* **Distance-wise Distillation:**
  * **개념:** Teacher 모델 공간에서 데이터 A와 B가 가깝고 C가 멀다면, Student 모델 공간에서도 A-B는 가깝고 C는 멀어야 합니다. 이러한 기하학적 구조를 보존합니다.


* **Graph-based Distillation:**
  * **개념:** 데이터를 그래프의 노드로 보고, Teacher가 형성한 그래프 구조(Graph Structure)를 Student가 모방하게 합니다.



### **4. LLM 시대의 KD 방법론 (Generative KD)**

최근 LLM 트렌드에서는 위 방법론들이 조금 다르게 적용됩니다. 특히 API만 열려있는 거대 모델(GPT-4, Solar 등)을 Teacher로 쓸 때는 **Black-box KD**가 주를 이룹니다.

* **CoT (Chain-of-Thought) Distillation:**
  * **개념:** 단순히 "답: A"를 배우는 게 아니라, Teacher가 생성한 **"추론 과정(Reasoning)"** 텍스트 자체를 학습 데이터에 포함시킵니다.
  * **효과:** Student 모델(소형 LLM)의 논리적 사고력을 비약적으로 상승시킵니다. 우리 실습의 프롬프트 엔지니어링 단계에서 `reason` 필드를 추가하는 것이 바로 이 효과를 노린 것입니다.


* **Instruction Tuning Distillation:**
  * **개념:** Teacher 모델을 이용해 다양한 지시문(Instruction)과 그에 대한 수행 결과 쌍을 대량 생성하여 Student를 튜닝합니다. (예: Stanford Alpaca)


* **In-Context Learning Distillation:**
  * **개념:** Teacher에게 Few-shot 예시를 주어 수행 능력을 끌어올린 뒤, 그 결과를 Student의 학습 데이터로 씁니다. 즉, Teacher의 '맥락 이해 능력'을 데이터 형태로 증류하는 것입니다.

#### 정리
1. 최근 LLM의 등장으로 인해서 현업에서는 과거의 모델 레이어를 건드리는 방법론에서 GPT 등의 거대 모델 API를 이용하여 합성 데이터를 만들거나 기존에 있는 데이터에 정답 라벨링을 모델로 만드는 방법론으로 전환하고 있습니다. 그 이유는 다음과 같습니다.
   1. 현업에서는 정제되지 않은 데이터는 매우 많습니다. 그러나 상대적으로 비싼 데이터 라벨링 비용으로 모든 데이터를 라벨링하지 못합니다.
   2. 레이어를 건드리는 효과 대비 고퀄리티 학습 데이터를 만드는 방법론이 좀더 효과적입니다. 특히, LLM으로 넘어오게 되면서 고퀄리티 학습 데이터의 중요성은 더더욱 커지고 있습니다.
2. Teacher 모델이 생성한 데이터를 학습함으로써 Teacher 모델의 정답을 Student 모델이 그대로 학습하게 되어 정답을 "전수"받는 효과를 얻습니다.

# 1. 환경설정 및 API 셋팅

- os environment variable에 대해서 알아봅니다.
- Solar API를 셋팅합니다.


### 1.1. os environment variable

🧾 환경 변수(Environment Variable)란?

환경 변수(Environment Variable)는 운영체제(Windows, macOS, Linux 등)가 프로그램 실행 환경에 제공하는 전역 설정값입니다.
프로그램은 이 환경 변수를 통해 중요한 정보(예: API 키, DB 비밀번호, 시스템 경로)를 읽을 수 있습니다.

1. 보안
- 코드 안에 직접 비밀번호나 API Key를 적으면 위험합니다.
- GitHub에 올리거나 다른 사람에게 코드 파일을 공유하면 키가 유출될 수 있습니다.
- 환경 변수는 운영체제 메모리에만 저장되므로, 코드 외부에서 안전하게 관리할 수 있습니다.

1. 유연성
- 같은 코드라도, 환경 변수만 바꾸면 다른 API 키나 설정을 사용할 수 있습니다.
- 예를 들어, 개발 서버와 배포 서버에서 다른 API 키를 사용하려면 코드 수정 없이 환경 변수만 변경하면 됩니다.

그러면 환경변수를 .env 파일에 저장하고, os.environ을 사용하여 환경변수를 읽는 방법을 사용해보도록 하겠습니다.

여기서 사용하는 환경변수는 API 키값이므로 API 키값을 받아와야 합니다.

API key는 [Upstage Console](https://console.upstage.ai/docs/getting-started)을 확인해주세요.

<img src="assets/img1.png" alt="example" style="max-width: 75%; height: auto;" />

<img src="assets/img2.png" alt="example" style="max-width: 75%; height: auto;" />

<img src="assets/img3.png" alt="example" style="max-width: 75%; height: auto;" />

.env.example 파일을 복사해서 .env로 만들고 그 파일 안에 Upstage API key를 넣어주세요.

저장된 .env 파일을 불러서 환경변수로 설정합니다.

In [ ]:
from google.colab import userdata
import os
import random

# tqdm이 텍스트 모드를 사용하도록 설정
os.environ["TQDM_DISABLE"] = "0"
os.environ["TQDM_MININTERVAL"] = "1"

# 시드 설정
random.seed(1234)

# 코랩 금고에서 API 키 꺼내오기! (dotenv 필요 없음)
UPSTAGE_API_KEY = userdata.get('UPSTAGE_API_KEY')

if UPSTAGE_API_KEY:
    print("Success API Key Setting! 🎉")
else:
    raise ValueError("API Key를 찾을 수 없습니다. 왼쪽 🔑 탭을 확인하세요.")

Success API Key Setting! 🎉


Upstage API를 셋팅했으니, 테스트로 llm 호출을 진행합니다.

Teacher 모델로 사용하기 위해서는 충분히 성능이 좋은 모델을 사용해야 합니다.

모든 Teacher 모델이 성능이 좋은 것은 아니지만, 일반적으로 모델 사이즈가 크다면 대체로 대부분의 영역에서 성능이 좋다고 알려져있기 때문에

우선 110B의 solar-pro3으로 진행합니다.

In [ ]:
from openai import OpenAI

client = OpenAI(api_key=UPSTAGE_API_KEY, base_url="https://api.upstage.ai/v1")

def chat_completion(prompt: str, model: str = "solar-pro3") -> str:
    resp = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ],
        stream=False,
    )
    return resp.choices[0].message.content

response = chat_completion("고양이에 대한 놀라운 사실 하나를 알려줘")
print(response)

**놀라운 사실:** 고양이는 사람보다 훨씬 높은 초음파까지 들을 수 있습니다.  
- 연구에 따르면 고양이는 약 **64 kHz**까지 주파수를 감지할 수 있는데, 이는 인간이 들을 수 있는 최고 주파수(≈20 kHz)보다 약 3배‑4배 더 높은 영역입니다.  
- 이렇게 넓은 청각 범위는 작은 설치류(예: 쥐, 들쥐)가 내는 고주파 소리를 포착하게 해, 사냥에 큰 이점을 줍니다.  

또한, 고양이의 귀에는 **32개의 작은 근육**이 있어 귀를 독립적으로 회전시킬 수 있습니다. 이 근육 덕분에 고양이는 소리의 방향을 아주 정밀하게 파악하고, 눈에 보이지 않는 소리까지 탐지할 수 있죠.  

이 두 가지 특징은 고양이가 왜 그렇게 뛰어난 사냥꾼이자, 주변 환경에 민감한 동물이 될 수 있는지를 설명해 줍니다. 🐾


# 2. 데이터 불러오기

- 다양한 Knowledge Distillation 방법론 중 Pseudo Labeling을 위한 Unlabeld Data를 불러옵니다.


In [ ]:
import pandas as pd

df = pd.read_csv("/train.csv") # Please adjust this path if you uploaded the file to a different location
df.head()

,db_id,question
0,department_management,How many heads of the departments are older th...
1,department_management,"List the name, born state and age of the heads..."
2,department_management,"List the creation year, name and budget of eac..."
3,department_management,What are the maximum and minimum budget of the...
4,department_management,What is the average number of employees of the...


데이터의 특성을 확인해봅니다.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4471 entries, 0 to 4470
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   db_id     4471 non-null   object
 1   question  4471 non-null   object
dtypes: object(2)
memory usage: 70.0+ KB


모든 데이터를 다 바꾸기엔 시간과 비용이 많이 들기 때문에 일부 데이터만 만들어보도록 하겠습니다.

In [ ]:
data = df[:10].to_dict(orient="records")
data

[{'db_id': 'department_management',
  'question': 'How many heads of the departments are older than 56 ?'},
 {'db_id': 'department_management',
  'question': 'List the name, born state and age of the heads of departments ordered by age.'},
 {'db_id': 'department_management',
  'question': 'List the creation year, name and budget of each department.'},
 {'db_id': 'department_management',
  'question': 'What are the maximum and minimum budget of the departments?'},
 {'db_id': 'department_management',
  'question': 'What is the average number of employees of the departments whose rank is between 10 and 15?'},
 {'db_id': 'department_management',
  'question': 'What are the names of the heads who are born outside the California state?'},
 {'db_id': 'department_management',
  'question': 'What are the names of the states where at least 3 heads were born?'},
 {'db_id': 'department_management',
  'question': "Show the name and number of employees for the departments managed by heads whose temp

# 3. 데이터 '생성'을 위한 Prompt Engineering

- 데이터 생성을 위한 Prompt Engineering에 대해 알아봅니다.
- 데이터 생성을 위한 API를 사용하여 데이터를 생성합니다.
- 생성된 데이터를 확인합니다.

### 3.1. 프롬프트 엔지니어링

<blockquote>
<b>🧠 프롬프트 엔지니어링(Prompt Engineering)이란?</b><br>
AI 모델에게 원하는 답을 정확하고 효율적으로 이끌어내기 위해 입력(프롬프트)을 설계·최적화하는 기술을 말합니다.
</blockquote>

1. 프롬프트 엔지니어링은 모델의 성능을 크게 좌우합니다.
	- AI 모델은 질문(프롬프트)에 따라 같은 데이터라도 완전히 다른 답을 내놓습니다.
		- 잘못된 프롬프트 → 모호하거나 엉뚱한 답변
		- 잘 설계된 프롬프트 → 정확하고 원하는 형식의 답변
	- 프롬프트 엔지니어링은 AI를 내 마음대로 컨트롤하는 리모컨과 같습니다.

2. 프롬프트의 기본 구조
하나의 프롬프트는 보통 다음 요소들로 구성됩니다.
	1. 역할(Role) 설정 → AI의 “정체성”과 “관점”을 지정합니다.
		- 예: “당신은 10년 경력의 여행 가이드입니다.”
	2. 목표(Task) 명확화 → 무엇을 해야 하는지 구체적으로 지시합니다.
		- 예: “서울에서 하루 동안 즐길 수 있는 코스를 추천해주세요.”
	3. 조건(Constraints) 부여 → 답변 길이, 형식, 언어, 스타일 등 제한사항을 명시합니다.
		- 예: “표 형식으로 3개의 추천 코스를 제시하고, 각 코스에 사진 링크를 포함하세요.”
	4.	예시(Examples) 제공 (Few-shot Prompting) → 원하는 답변 패턴을 보여주면 AI가 비슷하게 답변합니다.
		- 예: “예시: 코스 이름 - 설명 - 추천 이유”

3. 좋은 프롬프트 작성 원칙
	- 명확성 → 애매한 단어 대신 구체적인 지시
	- 맥락 제공 → 필요한 배경정보 포함
	- 출력 형식 지정 → 표, 목록, JSON 등
	- 예시 제시 → 원하는 결과물의 예시를 보여줌
	- 점진적 요청 → 복잡한 문제는 단계별로 쪼개서 요청
  
참고 [링크](https://www.notion.so/ssunbell/2341806f5bc1803491c3cea9d7a96933?source=copy_link)

### 3.2. JSON

<blockquote>
<b>🧠 JSON이란?</b><br>
JSON은 JavaScript Object Notation의 약자입니다.
</blockquote>

데이터를 저장하거나 주고받을 때 쓰는 가볍고 읽기 쉬운 데이터 형식입니다.
이름에 JavaScript가 들어있지만, Python, Java, C#, Go, Swift 등 대부분의 프로그래밍 언어에서 모두 사용할 수 있습니다.

1. 왜 JSON을 쓰나요?
	•	사람이 읽기 쉽다 → {}와 [], "키": 값 형태로 구조가 명확합니다.
	•	컴퓨터가 처리하기 쉽다 → 모든 언어에서 JSON을 쉽게 읽고 쓸 수 있는 라이브러리가 있음.
	•	언어에 독립적 → 어떤 언어에서 만든 JSON도 다른 언어에서 그대로 읽을 수 있음.
	•	웹 표준 → API, 서버·클라이언트 데이터 통신에 가장 널리 쓰임.

2. JSON의 기본 문법
객체(Object) : {} 안에 "키": 값 쌍으로 구성되어 있습니다.
```json
{
  "name": "Alice",
  "age": 25,
  "is_student": false
}
```
3. 모델의 structured output에서는 보통 JSON의 형식을 많이 사용합니다.
- LLM 모델은 지정된 형식을 따라 generate할 수 있도록 만들 수 있는데, 이때 대표적으로 사용하는 형식이 JSON 형식입니다.

아래 코드를 실행하여 어떤 방식으로 JSON 형식으로 만드는지 확인해봅니다.

In [ ]:
import json

user_prompt = """가상의 데이터를 만들려고 합니다.

아래 output_format에 따라 JSON 데이터를 1개만 반환해주세요.
{output_format}
"""

output_format ="""<output_format>
```json
{
    "name": [이름],
    "age": [나이]
    "is_student": [true 또는 false]
}
```
</output_format>"""

messages=[
    {
        "role": "user",
        "content": user_prompt.format(output_format=output_format)
    }
]

response = client.chat.completions.create(
    model="solar-pro3",
    messages=messages,
)

def json_parsing(output_text:str) -> dict:
    output_text = output_text[output_text.index("```json") + len("```json"):].strip()
    output_text = output_text[:output_text.index("```")].strip()
    return json.loads(output_text)


output = response.choices[0].message.content
print("현재 응답의 형식: ", type(output))
print(output)
structured_dictionary = json_parsing(output)
print("구조화된 응답의 형식: ", type(structured_dictionary))
print(structured_dictionary)

현재 응답의 형식:  <class 'str'>
```json
{
    "name": "김하늘",
    "age": 24,
    "is_student": false
}
```
구조화된 응답의 형식:  <class 'dict'>
{'name': '김하늘', 'age': 24, 'is_student': False}


<blockquote>
<b>🧠 response_format이란?</b><br>
prompt에 json 형식의 output을 정의하지 않고 response_format에 직접 json schema를 정의해서 output format을 정의할 수도 있습니다.
</blockquote>

1. 프롬프트에 어떤 output format을 지정하게 되면 응답 결과물에 추가로 parsing 함수를 정의해서 json으로 변환 가능한 텍스트만 추출해야 합니다.
2. 만약, 모델의 성능이 떨어지는 상횡이 생기면 parsing이 안될 가능성도 존재합니다.
3. 따라서, 좀더 안정적인 결과물을 얻기 위해서는 `response_format`을 사용하는 것이 좋습니다.

자세한 사용법은 [openai structured output](https://platform.openai.com/docs/guides/structured-outputs?lang=python)에 대한 문서를 참고해주세요.


In [ ]:
from openai import OpenAI
import json


messages=[
    {
        "role": "system",
        "content": "You are an expert in information extraction. Extract information from the given HTML representation of image and organize them into a clear and accurate JSON format."
    },
    {
        "role": "user",
        "content": "HTML string: <table id='0' style='font-size:14px'><tr><td>1</td><td>FUTAMI 17 GREEN TEA (CLAS</td><td>12,500</td></tr><tr><td>1</td><td>EGG TART</td><td>13,000</td></tr><tr><td>1</td><td>GRAIN CROQUE MONSIEUR</td><td>17,000</td></tr></table><br><table id='1' style='font-size:18px'><tr><td>TOTAL</td><td>42, 500</td></tr><tr><td>CASH</td><td>50,000</td></tr><tr><td></td><td></td></tr><tr><td>CHANGE</td><td>7 ,500</td></tr></table>\n. Extract the structured data from the HTML string in JSON format."
    }
]

response_format={
    "type": "json_schema",
    "json_schema": {
        "name": "restaurant_receipt",
        "strict": True,
        "schema": {
            "type": "object",
            "properties": {
                "menu_items": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "menu_cnt": {
                                "type": "number",
                                "description": "The count of the menu item."
                            },
                            "menu_name": {
                                "type": "string",
                                "description": "The name of the menu item."
                            },
                            "menu_price": {
                                "type": "number",
                                "description": "The price of the menu item."
                            }
                        },
                        "required": ["menu_cnt", "menu_name", "menu_price"],
                    }
                },
                "total_price": {
                    "type": "number",
                    "description": "The total price of the receipt."
                }
            },
            "required": ["menu_items", "total_price"],
        }
    }
}

response = client.chat.completions.create(
    model="solar-pro3",
    messages=messages,
    response_format=response_format
)

structured_output = response.choices[0].message.content
print("현재 응답의 형식: ", type(structured_output))
print(structured_output)
structured_dictionary = json.loads(structured_output)
print("구조화된 응답의 형식: ", type(structured_dictionary))
print(structured_dictionary)

현재 응답의 형식:  <class 'str'>
{
  "menu_items": [
    {
      "menu_cnt": 1,
      "menu_name": "FUTAMI 17 GREEN TEA (CLAS",
      "menu_price": 12500
    },
    {
      "menu_cnt": 1,
      "menu_name": "EGG TART",
      "menu_price": 13000
    },
    {
      "menu_cnt": 1,
      "menu_name": "GRAIN CROQUE MONSIEUR",
      "menu_price": 17000
    }
  ],
  "total_price": 42500
}
구조화된 응답의 형식:  <class 'dict'>
{'menu_items': [{'menu_cnt': 1, 'menu_name': 'FUTAMI 17 GREEN TEA (CLAS', 'menu_price': 12500}, {'menu_cnt': 1, 'menu_name': 'EGG TART', 'menu_price': 13000}, {'menu_cnt': 1, 'menu_name': 'GRAIN CROQUE MONSIEUR', 'menu_price': 17000}], 'total_price': 42500}


확인하신 것처럼 JSON으로 변환이 가능한 string 타입의 응답을 반환합니다. 해당 응답을 `json.loads()`를 사용하여 dictionary 타입으로 변환할 수 있습니다.

만약, 모델의 성능이 떨어지게 되면 JSON으로 변환이 불가능한 string 타입으로 반환할 수 있으므로 `try except문`을 통해 에러 처리를 해주시면 좋습니다.

### 3.3. reasoning

#### 1. 핵심 개념: "생각의 시간" (Thinking Budget)

기존 LLM(GPT-4, Gemini 1.5 Flash 등)은 기본적으로 "System 1 (직관적 사고)"에 가깝습니다. 질문을 받으면 즉시 답변 토큰을 생성하기 시작합니다.

반면, `reasoning_effort`가 적용된 모델은 "System 2 (분석적 사고)"를 시뮬레이션합니다. 답변을 출력하기 전에 보이지 않는 내부 사고 과정(Internal Chain of Thought)**을 거칩니다. `reasoning_effort`는 바로 이 **내부 사고의 깊이와 시간(Budget)을 조절하는 레버입니다.

#### 2. 설정 값에 따른 차이 (Low vs. Medium vs. High)

보통 3단계(Low, Medium, High)로 나뉘며, 엔지니어링 관점에서는 다음과 같은 차이가 있습니다.

##### **Low (낮음/빠름)**

* **동작:** 모델이 문제를 보자마자 가장 먼저 떠오른 직관적인 해결책을 채택합니다. 내부 검증 과정을 최소화합니다.
* **특징:** 응답 속도가 빠르고 비용이 저렴합니다.
* **용도:** 단순 번역, 요약, 문법 교정, 감성 분석 등 복잡한 논리가 필요 없는 작업.

##### **Medium (중간/표준)**

* **동작:** 표준적인 사고 과정을 거칩니다. 문제를 분해하고, 계획을 세우고, 답변을 생성합니다.
* **용도:** 일반적인 코딩 질문, 이메일 작성, 맥락 파악이 필요한 대화.

##### **High (높음/깊음)**

* **동작:** 모델이 답변을 확정하기 전에 다양한 경로를 탐색하고, 스스로 오류를 검증(Self-Correction)하며, 엣지 케이스(Edge Case)를 고민합니다.
* **특징:** 응답 속도가 현저히 느려질 수 있고(수 초~수십 초 생각), 토큰 비용이 비쌉니다.
* **용도:**
  * 복잡한 수학 증명
  * 고난이도 알고리즘 코딩 (예: "이 코드를 O(n) 복잡도로 최적화해줘")
  * 복잡한 스키마를 가진 Text-to-SQL (우리의 현재 타겟)
  * 과학적 추론

#### 3. Text-to-SQL 작업에서의 적용

* **언제 `High`를 써야 할까요?**
  * 테이블이 5개 이상 조인(Join)되어야 할 때.
  * `HAVING`, 서브쿼리(Subquery), `Self-Join` 같은 복잡한 논리가 필요할 때.
  * 질문이 모호해서 스키마의 여러 컬럼 중 무엇을 매핑해야 할지 깊은 추론이 필요할 때.
  * *예: "지난 3년 동안 분기별 매출 성장률이 가장 높았던 부서의 매니저 이름은?"*


* **언제 `Low/Medium`이 유리할까요?**
  * 단순한 `SELECT * FROM table WHERE ...` 수준의 쿼리일 때.
  * 대량의 데이터를 빠르게 라벨링해야 하는데 예산(API Cost)이 제한적일 때.

따라서 일반적인 LLM보다 reasoning 모델이 좀더 좋은 성능을 가져옵니다. 이러한 reasoning model은 작은 사이즈의 모델은 잘 없기 때문에 Teacher 모델로 적절하게 사용할 수 있습니다.

아래 reasoning 예시를 보면서 이해해봅니다.

In [ ]:
stream = client.chat.completions.create(
    model="solar-pro3",
    messages=[
        {
            "role": "user",
            "content": "What is 15% of 80?"
        }
    ],
    reasoning_effort="high",
    stream=True,
)

reasoning_started = False
content_started = False

for chunk in stream:
    delta = chunk.choices[0].delta

    # Handle reasoning output (thinking process)
    if hasattr(delta, 'reasoning') and delta.reasoning:
        if not reasoning_started:
            print("[Reasoning]")
            reasoning_started = True
        print(delta.reasoning, end='')

    # Handle content output (final answer)
    if hasattr(delta, 'content') and delta.content:
        if not content_started:
            print("\n[Answer]")
            content_started = True
        print(delta.content, end='')

[Reasoning]
The user asks: "What is 15% of 80?" This is a simple arithmetic question. Provide answer: 12. Show steps: 15% = 0.15, multiply by 80 => 12.

We can respond concisely: 15% of 80 = 12.

But we can also add explanation: 15% = 15/100 = 3/20, 80 * 3/20 = 240/20 = 12.

We can answer. Possibly also note that 15% of 80 is 12.

Thus final answer: 12.

We can also ask if they need more help.

So final answer: 12.

That's straightforward.

We should comply with instructions: no extraneous content. Provide answer.

We can also mention that 15% of 80 = 12.

Okay.
[Answer]
15% of 80 is **12**.

**Explanation:**  
15 % = 0.15 = 15⁄100 = 3⁄20.  
Multiplying by 80:  

\[
80 \times \frac{3}{20} = \frac{80 \times 3}{20} = \frac{240}{20} = 12
\]

So, 15 % of 80 equals 12. If you need further help with percentages or any other calculations, feel free to ask!

- delta.reasoning : LLM이 생각한 사고 과정(CoT)를 나타냅니다.
- delta.content : LLM이 생각한 사고를 바탕으로 만들어낸 응답

그러면 실제로 시스템 프롬프트를 다르게 작성했을때 어떻게 답변 퀄리티가 달라지는지 확인해보겠습니다.

In [ ]:
GENERATOR_SYSTEM_PROMPT = """### 1. ROLE
You are a World-Class SQL Architect and Data Scientist specialized in translating natural language questions into complex SQLite queries. You have a deep understanding of database normalization, join operations, and logical filtering. You are meticulous about adhering strictly to provided database schemas.

### 2. TASK
Your task is to generate a syntactically correct and semantically accurate SQLite query based on the provided user question and database schema.
You must analyze the relationship between tables, identify the correct columns for SELECT, WHERE, GROUP BY, and ORDER BY clauses, and construct a query that exactly answers the question.

### 3. CONSTRAINTS
- **Dialect:** Use standard SQLite syntax.
- **Schema Adherence:** purely strictly base your query on the provided table names and column names. Do NOT assume the existence of any columns not explicitly mentioned in the schema context.
- **Output Format:** Provide ONLY the SQL query code. Do not include any explanations, markdown formatting (like ```sql), or conversational text.
- **Table Aliasing:** Use clear aliases (e.g., T1, T2) for tables when performing JOIN operations to avoid ambiguity.
- **Reasoning:** Ensure the logic handles aggregation (COUNT, MAX, MIN, AVG) and conditional logic (AND, OR, NOT) correctly as implied by the natural language question.

### 4. EXAMPLES (Few-shot Prompting)

Input:
Question: What are the names of the heads who are born in California?

Output:
SELECT name FROM head WHERE born_state = 'California'

Input:
Question: Find the first name and age of students who are enrolled in 'Data Structures'.

Output:
SELECT T1.FName, T1.Age FROM student AS T1 JOIN enroll AS T2 ON T1.St_ID = T2.St_ID JOIN class AS T3 ON T2.Class_Code = T3.Class_Code WHERE T3.Class_Name = 'Data Structures'

Input:
Question: Show the name and theme of all concerts and the number of singers in each concert.

Output:
SELECT T2.Concert_Name, T2.Theme, COUNT(*) FROM singer_in_concert AS T1 JOIN concert AS T2 ON T1.Concert_ID = T2.Concert_ID GROUP BY T2.Concert_ID
"""

stream = client.chat.completions.create(
    model="solar-pro3",
    messages=[
        {
            "role": "system",
            "content": GENERATOR_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": data[0]["question"]
        }
    ],
    reasoning_effort="high",
    stream=True,
)

reasoning_started = False
content_started = False

for chunk in stream:
    delta = chunk.choices[0].delta

    # Handle reasoning output (thinking process)
    if hasattr(delta, 'reasoning') and delta.reasoning:
        if not reasoning_started:
            print("[Reasoning]")
            reasoning_started = True
        print(delta.reasoning, end='')

    # Handle content output (final answer)
    if hasattr(delta, 'content') and delta.content:
        if not content_started:
            print("\n[Answer]")
            content_started = True
        print(delta.content, end='')

[Reasoning]
We need to parse the user's question: "How many heads of the departments are older than 56 ?" So we need to count heads of departments older than 56. The schema presumably includes tables: head, department, maybe relationship. However we need to consider the context: The system is a World-Class SQL Architect specialized in translating natural language questions into complex SQLite queries. We need to adhere strictly to provided database schemas. The user hasn't provided a schema. The system might have hidden context: The schema is known to the model? Usually these tasks include a schema provided earlier in conversation. However we have no explicit schema in the conversation. The system says "You must analyze the relationship between tables, identify the correct columns for SELECT, WHERE, GROUP BY, and ORDER BY clauses, and construct a query that exactly answers the question." There's no schema. We need to infer typical naming conventions for a head of department database. P

### 4.1. Pseudo Labeling

<blockquote>
<b>🧠 수도 라벨링(Pseudo Labeling)란?</b><br>
수도 라벨링은 사람이 아닌 모델 혹은 알고리즘으로 생성한 정답 데이터입니다.
</blockquote>

수도 라벨링을 이용한 KD는 아래와 같은 목적으로 사용될 수 있습니다.

1.  정답 일관성
	- Teacher 모델의 응답은 다양한 사람들보다는 어느정도 일관성을 가지고 정답을 라벨링하게 되어 수많은 사람들의 데이터를 모아 정규분포를 만들 필요 없이 한 모델로 일관성을 갖출 수 있습니다.
	- 심지어는 사람보다 더 나은 결과물을 보이는 경우가 있기 때문에 일반 라벨러보다 더 좋은 성능을 보여줄 수 있습니다.

2.  비용 절감
	- 초거대 LLM을 학습하기 위해서는 Google, OpenAI 등 천문학적인 비용이 들게 됩니다.
	- 천문학적인 돈을 들여서 만든 모델의 정답 데이터를 학습시킬 수 있으므로 비용을 매우 절감할 수 있습니다.
	- ex : deepseek-R1 모델을 학습시킬때 GPT의 데이터를 추출하여 학습시켰다는 논란이 있었습니다. 이처럼 Teacher 모델의 지식을 학습시키는 방법론은 일반적으로 많이 수행하는 방법론입니다.
  
그러면 실제로 수도 라벨링을 하여 정답 데이터를 가져오도록 합니다.

In [ ]:
def chat_completion(user_prompt: str, system_prompt: str | None = None, model: str = "solar-pro3") -> tuple[str, str]:
    messages = []
    if system_prompt:
        messages.append({
            "role": "system",
            "content": system_prompt
        })
    messages.append({
        "role": "user",
        "content": user_prompt
    })

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        reasoning_effort="high",
        stream=False,
    )
    return response.choices[0].message.content, response.choices[0].message.reasoning # [answer, reasoning]

위와 같이 함수를 정의했을 때, 반환값은 2가지 입니다.

1. 응답값 : response.choices[0].message.content
2. 생각 과정 : response.choices[0].message.reasoning

이 경우 2. 생각 과정을 검증하여 응답값이 적절한지 다시 판단하는 LLM as a Judge에 활용할 수 있습니다.

해당 값들을 포함하여 데이터를 만들어봅니다.

추론 과정이 매우 길기 때문에 시간이 오래 걸립니다.

In [ ]:
from tqdm.auto import tqdm

labeled_data = []
for i in tqdm(range(len(data))):
    record = {**data[i]}
    question = data[i]["question"]
    answer, reasoning = chat_completion(question, GENERATOR_SYSTEM_PROMPT, "solar-pro3")
    record["answer"] = answer
    record["reasoning"] = reasoning
    labeled_data.append(record)

print(labeled_data)

  0%|          | 0/10 [00:00<?, ?it/s]

[{'db_id': 'department_management', 'question': 'How many heads of the departments are older than 56 ?', 'answer': 'SELECT COUNT(*) FROM head AS h WHERE h.Age > 56', 'reasoning': 'The user asks: "How many heads of the departments are older than 56 ?" We need to generate a SQLite query based on provided schema. The user hasn\'t provided the schema directly in the conversation, but the system prompt says we must base on provided table names and column names. However, we haven\'t been given any schema. The user question implies there is a table for "heads of the departments". Possibly a table named "head" or "department_head". The schema might be known from prior context? The system prompt says we must adhere strictly to provided database schemas. But we haven\'t seen them. In the absence of schema, we need to infer typical naming. The examples used "head" table with columns "name" and "born_state". So likely there is a "head" table with columns like "Head_ID", "Dept_ID", "Age", maybe "De

### 4.2. LLM as a Judge

<blockquote>
<b>🧠 LLM as a Judge란?</b><br>
사람 대신 LLM 모델을 이용하여 정성적인 결과물을 평가하는 것을 의미합니다.
</blockquote>

1. LLM as a Judge라는 말은 LLM이 판사처럼 다른 답변을 평가한다는 뜻입니다.
- 어떤 질문에 대해 여러 학생이 답을 했다고 해본다고 하는 경우에
- 그러면 LLM이 각 답을 읽고, “이 답은 정확한가?”, “논리가 맞는가?”, “설명이 충분한가?” 같은 기준으로 점수를 매깁니다.

2. 사용 이유
- 자동 채점: 시험 문제나 과제를 사람이 일일이 채점하는 대신, LLM이 먼저 평가를 도와줄 수 있습니다. (비용 효율적)
- 피드백 제공: 단순히 점수만 주는 게 아니라, 어떤 부분이 좋고 어떤 부분이 부족한지 설명까지 덧붙일 수 있습니다.

3. 장점과 한계
- 장점: 빠르고, 일관성 있게 많은 답을 평가할 수 있습니다.
- 한계: LLM은 편향이 존재하기 때문에 완벽하지 않습니다. SQL문 처럼 어느정도 정답군이 존재할 경우 LLM as a Judge보다 코드 실행 평가가 더욱 효과가 클 수 있습니다.

4. 사용 원칙
- 평가 결과물은 점수(score)와 이유(reason)로 구성됩니다. reason을 생성하게 함으로써 모델이 좀더 정확한 평가를 하게 만듭니다.
- 평가 결과물이 일관되도록 seed 설정과 temperature를 0으로 설정합니다.

여기서는 다른 데이터셋에서도 활용이 가능한 LLM as a Judge 방법론을 경험해봅니다.

그러면 LLM as a Judge를 이용하여 이전에 생성한 합성 데이터 결과물을 평가해보겠습니다.

In [ ]:
evaluation_response_format = {
    "type": "json_schema",
    "json_schema": {
        "name": "evaluation_result",
        "strict": True, # 모델이 스키마를 강제로 따르도록 설정
        "schema": {
            "type": "object",
            "properties": {
                "score": {
                    "type": "integer",
                    "description": "An integer score between 1 and 5 indicating the quality of reasoning.",
                    # 1~5 범위 강제 (지원 모델의 경우)
                    "minimum": 1,
                    "maximum": 5
                },
                "critique": {
                    "type": "string",
                    "description": "A detailed explanation of the score, focusing on logical consistency and correctness."
                }
            },
            "required": ["score", "critique"],
            "additionalProperties": False
        }
    }
}

JUDGE_SYSTEM_PROMPT= """# Role
You are an Expert Logic Verifier and AI Evaluator. Your goal is to critically assess a model's performance based on its "Reasoning Process" (Chain of Thought) and "Final Response".

# Evaluation Criteria
You must evaluate the submission based on the following strict criteria:

1. **Logical Consistency**: Does the reasoning flow logically from step to step? Are there any gaps, jumps, or contradictions?
2. **Correctness**: Is the final answer factually and mathematically correct?
3. **Alignment**: Does the final answer directly result from the reasoning process provided? (Penalize if the reasoning is wrong but the answer is luckily correct).
4. **Efficiency**: Does the model avoid unnecessary verbosity and repetition? (Do not give higher scores just for longer text).

# Scoring Scale (1-5)
- **5 (Perfect)**: Flawless reasoning, correct answer, efficient explanation.
- **4 (Good)**: Correct answer with valid reasoning, but contains minor inefficiencies or redundant steps.
- **3 (Acceptable)**: Correct answer, but the reasoning is shaky, incomplete, or contains minor logical slips that don't affect the final outcome.
- **2 (Flawed)**: Wrong answer due to logical errors, OR correct answer derived from completely incorrect reasoning (Hallucinated Logic).
- **1 (Failure)**: Completely incorrect reasoning and incorrect answer, or irrelevant content.

# Input Format
- Reasoning Process: {reasoning}
- Final Response: {answer}

# Output Format
Provide your assessment in the following JSON format strictly:
{
  "score": "<integer_between_1_and_5>",
  "critique": "<string_explanation_of_the_score_focusing_on_logical_flaws>"
}
"""
user_prompt = "- Reasoning Process:" + labeled_data[0]["reasoning"] + "\n" + "- Final Response:" + labeled_data[0]["answer"]

response = client.chat.completions.create(
    model="solar-pro3",
    messages=[
        {
            "role": "system",
            "content": JUDGE_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": user_prompt
        }
    ],
    response_format=evaluation_response_format,
    seed=42,
    temperature=0
)

llm_as_judge_output = response.choices[0].message.content
json.loads(llm_as_judge_output)

{'score': 5,
 'critique': "The reasoning process logically deduces that the task requires counting heads older than 56, correctly assumes the existence of a 'head' table with an 'Age' column, and selects an appropriate concise query. The final response aligns perfectly with the reasoning, providing a correct and efficient SQLite statement without unnecessary verbosity. No logical gaps, contradictions, or hallucinations are present."}

이제 검수하기 위해서 LLM as a Judge 결과물을 보고 비교 평가를 진행합니다.

아무리 LLM의 성능이 좋더라고 어느정도 사람의 검수가 필요한 부분이 있습니다.

특히, 해당 영역에 전문가라면 LLM보다 더 높은 수준의 지식을 알거나 라벨링을 많이 해본 라벨러라면 만든 데이터의 특성이 이상하다는 것을 판단할 수 있습니다.

그렇기에 LLM as a Judge는 어느정도 판단의 기준으로만 사용하고 사람의 검수를 하는 것이 가장 고퀄리티로 맞출 수 있는 방법입니다.

그러면 이제 LLM as a Judge 결과물을 담고 파일로 만들어서 직접 검수를 진행해보시길 바랍니다.

In [ ]:
labeled_data_with_judge = []
for i in tqdm(range(len(labeled_data))):
    user_prompt = (
        "- Reasoning Process:"
        + labeled_data[i]["reasoning"]
        + "\n"
        + "- Final Response:"
        + labeled_data[i]["answer"]
    )

    response = client.chat.completions.create(
        model="solar-pro3",
        messages=[
            {"role": "system", "content": JUDGE_SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt},
        ],
        response_format=evaluation_response_format,
        seed=42,
        temperature=0,
    )

    llm_as_judge_output = response.choices[0].message.content
    labeled_data_with_judge.append({
        **labeled_data[i],
        **json.loads(llm_as_judge_output),
    })

print(labeled_data_with_judge)

  0%|          | 0/10 [00:00<?, ?it/s]

[{'db_id': 'department_management', 'question': 'How many heads of the departments are older than 56 ?', 'answer': 'SELECT COUNT(*) FROM head AS h WHERE h.Age > 56', 'reasoning': 'The user asks: "How many heads of the departments are older than 56 ?" We need to generate a SQLite query based on provided schema. The user hasn\'t provided the schema directly in the conversation, but the system prompt says we must base on provided table names and column names. However, we haven\'t been given any schema. The user question implies there is a table for "heads of the departments". Possibly a table named "head" or "department_head". The schema might be known from prior context? The system prompt says we must adhere strictly to provided database schemas. But we haven\'t seen them. In the absence of schema, we need to infer typical naming. The examples used "head" table with columns "name" and "born_state". So likely there is a "head" table with columns like "Head_ID", "Dept_ID", "Age", maybe "De

In [ ]:
with open("labeled_data_with_judge.json", "w", encoding="utf-8-sig") as f:
    json.dump(labeled_data_with_judge, f, indent=4, ensure_ascii=False)


지금까지 Knowledge Distillation을 위해서 기본 환경 설정과 프롬프트 엔지니어링, LLM as a Judge 방식을 배워보았습니다.

모델의 성능에 따라서 비용도 증가하고 시간도 증가하기 때문에 어떻게 하면 비용과 시간을 최적화하여 데이터를 만들 수 있을지 고민해보시길 바랍니다.
